# Define Imports

In [2]:
import os
import whisper
import subprocess
import json
from ollama import chat
from ollama import ChatResponse
import pandas as pd

/home/vinic/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Transcribe Audio

In [ ]:
def transcribe_audio(audio_file):
    model = whisper.load_model("base")
    audio = whisper.load_audio(audio_file,sr=16000)
    audio_tensor = torch.from_numpy(audio).to(torch.float32)
    result = model.transcribe(audio_tensor, fp16=False)['text']
    return result

def listfiles():
    arr = os.listdir("ligacoes")
    return arr

files = listfiles()

for file in files:
    if(file.split(".")[1]=="mp3"):
       subprocess.check_call(["whisper", "/home/vinic/development/python/DasaDados/ligacoes/" + file, "--language=Portuguese", "--fp16=False"]) 

#transcribe_audio(audio)

# GenAI Analysis

#### Initial Analisys   

In [ ]:
def listfiles():
    arr = os.listdir("transcricoes")
    return arr

with open('config.json') as json_data:
    config = json.load(json_data)   

text_files = listfiles()
responses = []

for file in text_files:

    print("processando " + file)

    if( len(file.split(".mp3."))>1 and file.split(".mp3.")[1]=="txt"):

        with open("transcricoes/" + file) as f: filetext = f.read()
        json_obj={}

        # Chamada GenAI com Prompt Inicial
        content = config["prompt_inicial"].replace("[[transcricao]]", "[[" + filetext + "]]")[:5000]
        response: ChatResponse = chat(model='llama3.2', messages=[
            {
                'role': 'user',
                'content': content
            },
        ])
        #print(response.message.content)
        
        try:
            json_obj = json.loads(response.message.content)
            json_obj["file_name"] = file
            json_obj["token_in"] = response.prompt_eval_count
            json_obj["token_out"] = response.eval_count

        except:
            f = open("error.txt", "a")
            f.write(file)
            f.write(response.message.content)
            f.close()

        # Caso seja agendamento, validar informações adicionais
        if( "tipo" in json_obj and json_obj["tipo"]=="Agendamento"):

            json_obj_agendamento={}

            # Chamada GenAI com Prompt de Agendamento
            content = config["prompt_agendamento"].replace("[[transcricao]]", "[[" + filetext + "]]")[:5000]
            response: ChatResponse = chat(model='llama3.2', messages=[
                {
                    'role': 'user',
                    'content': content
                },
            ])
            #print(response.message.content)
            
            try:
                json_obj_agendamento = json.loads(response.message.content)

                json_obj["tipo_atendimento"]=json_obj_agendamento["tipo_atendimento"]
                json_obj["nome_convenio"]=json_obj_agendamento["nome_convenio"]
                json_obj["laboratorio"]=json_obj_agendamento["laboratorio"]
                json_obj["oferta"]=json_obj_agendamento["oferta"]

            except:
                f = open("error.txt", "a")
                f.write(file)
                f.write(response.message.content)
                f.close()

        responses.append(json_obj)
        df = pd.DataFrame(responses)
        df.to_csv('output.csv', index=False)
        

#### Exam Extraction

In [ ]:
def listfiles():
    arr = os.listdir("transcricoes")
    return arr

with open('config.json') as json_data:
    config = json.load(json_data)   

text_files = listfiles()
responses = []
df = pd.read_csv('output.csv')

for file in text_files:

    print("processando " + file)

    # Find item classification on CSV
    item_output_csv = df.loc[df["file_name"]==file, ["tipo"]]
    
    # Process items identified as AGENDAMENTO
    if( len(file.split("."))>1 and file.split(".mp3.")[1]=="txt" and  len(item_output_csv) > 0 and item_output_csv.iloc[0]["tipo"]=="Agendamento"):
        print("Agendamento, processo iniciado")

        with open("transcricoes/" + file) as f: filetext = f.read()
        json_obj={}

        # Chamada GenAI com Prompt de Exames
        content = config["prompt_agendamento_exames"].replace("[[transcricao]]", "[[" + filetext + "]]")[:5000]
        response: ChatResponse = chat(model='llama3.2', messages=[
            {
                'role': 'user',
                'content': content
            },
        ])
        #print(response.message.content)
        
        try:
            array_exames = response.message.content.split(",")
            
            for exame in array_exames:
                f = open("output_exames.csv", "a")
                f.write(file + ',"' +  exame.replace("\n", "") + '", ' + str(response.prompt_eval_count) + ', ' + str(response.eval_count)  + '\n')
                f.close()


        except:
            f = open("error.txt", "a")
            f.write(file)
            f.write(response.message.content)
            f.close()

       

#### Schedule Fail Reason

In [ ]:
def listfiles():
    arr = os.listdir("transcricoes")
    return arr

with open('config.json') as json_data:
    config = json.load(json_data)   

text_files = listfiles()
responses = []
df = pd.read_csv('output.csv')

for file in text_files:

    print("processando " + file)

    # Find item classification on CSV
    item_output_csv = df.loc[df["file_name"]==file, ["tipo", "resolvido"]]
    
    # Process items identified as AGENDAMENTO
    if( len(file.split("."))>1 and file.split(".mp3.")[1]=="txt" and  len(item_output_csv) > 0 and item_output_csv.iloc[0]["tipo"]=="Agendamento" and item_output_csv.iloc[0]["resolvido"]==False):
        print("Agendamento, processo iniciado")

        with open("transcricoes/" + file) as f: filetext = f.read()
        json_obj={}

        # Chamada GenAI com Prompt de Exames
        content = config["prompt_agendamento_motivo_negativo"].replace("[[transcricao]]", "[[" + filetext + "]]")[:5000]
        response: ChatResponse = chat(model='llama3.2', messages=[
            {
                'role': 'user',
                'content': content
            },
        ])
        #print(response.message.content)
        
        try:
            f = open("output_schedule_false.csv", "a")
            f.write(file + ',"' +   response.message.content   + '"\n')
            f.close()


        except:
            f = open("error.txt", "a")
            f.write(file)
            f.write(response.message.content)
            f.close()

       